# 2020 US Presidential Elections Winner Simulator
[PredictIt](https://www.predictit.org/) is a financial prediction market for various political events around the world. 
This simulator attempts to find how often Joe Biden will win the 2020 US Election as inferred from the state-by-state markets on PredictIt.

## Download Electoral Votes Data
Get the number of electoral votes assigned to each congressional district.

In [130]:
import csv
import io
import urllib.request

url_open = urllib.request.urlopen("https://raw.githubusercontent.com/peterhhchan/us-elections-2020/master/states.csv")
reader = csv.reader(io.TextIOWrapper(url_open, encoding = 'utf-8'), delimiter=',')

electoral_votes = {}
for v,n,a in reader:
    electoral_votes[a]=int(v)
    
print (electoral_votes)

{'CA': 55, 'TX': 38, 'FL': 29, 'NY': 29, 'IL': 20, 'PA': 20, 'OH': 18, 'GA': 16, 'MI': 16, 'NC': 15, 'NJ': 14, 'VA': 13, 'WA': 12, 'AZ': 11, 'IN': 11, 'MA': 11, 'TN': 11, 'MD': 10, 'MN': 10, 'MO': 10, 'WI': 10, 'AL': 9, 'CO': 9, 'SC': 9, 'KY': 8, 'LA': 8, 'CT': 7, 'OK': 7, 'OR': 7, 'AR': 6, 'IA': 6, 'KS': 6, 'MS': 6, 'NV': 6, 'UT': 6, 'NE': 5, 'NM': 5, 'WV': 5, 'HI': 4, 'ID': 4, 'ME': 4, 'NH': 4, 'RI': 4, 'AK': 3, 'DE': 3, 'MT': 3, 'ND': 3, 'SD': 3, 'VT': 3, 'WY': 3, 'DC': 3}


## Download Market Data
Grab the latest share prices on PredictIt.

In [152]:
import json
import re

prices = {}
markets = json.loads(urllib.request.urlopen("https://www.predictit.org/api/marketdata/all/").read())['markets']
for m in markets:
    sn = m['shortName']
    match = re.search("^Which party will win ([A-Z]{2})( in)? 2020\?$", sn)
    if match:
        for c in m['contracts']:
            if c['name'] == 'Democratic':
                prices[match[1]] = c['lastTradePrice']

print (prices)

{'WI': 0.74, 'PA': 0.73, 'FL': 0.56, 'MI': 0.78, 'AZ': 0.7, 'MN': 0.79, 'NH': 0.8, 'NC': 0.53, 'OH': 0.45, 'NV': 0.79, 'VA': 0.92, 'IA': 0.42, 'GA': 0.44, 'CO': 0.9, 'TX': 0.34, 'ME': 0.89, 'IN': 0.1, 'NM': 0.91, 'NJ': 0.95, 'MO': 0.14, 'OR': 0.94, 'UT': 0.08, 'TN': 0.08, 'CT': 0.95, 'AK': 0.23, 'KY': 0.06, 'MD': 0.96, 'MA': 0.97, 'AR': 0.07, 'WA': 0.95, 'MT': 0.14, 'SC': 0.19, 'CA': 0.95, 'NY': 0.95, 'IL': 0.95, 'WV': 0.05, 'OK': 0.04, 'LA': 0.09, 'ID': 0.03, 'NE': 0.07, 'AL': 0.05, 'KS': 0.11, 'MS': 0.09, 'RI': 0.96, 'HI': 0.95, 'WY': 0.04, 'VT': 0.95, 'DE': 0.96, 'ND': 0.04, 'SD': 0.07, 'DC': 0.98}


## Calculate the Implied Probability
PredictIt charges a 10% fee on profits, the savvy bettor understands that in order to break-even, a 50-cent contract must win more than 52.6% of the time.

In [153]:
win_prob = {}
for s, price in prices.items():
    p = float (price)
    ## predictit charges a 10% fee on profits
    ## the net profit on a 50 cent contract is 45 cents
    ## the bettor must win 52.63%+ of the time to breakeven
    win_prob[s] = p / ((1 - p) * 0.9 + p)
print (win_prob)

{'WI': 0.7597535934291582, 'PA': 0.750256937307297, 'FL': 0.5857740585774059, 'MI': 0.7975460122699387, 'AZ': 0.7216494845360825, 'MN': 0.8069458631256384, 'NH': 0.8163265306122449, 'NC': 0.5561385099685204, 'OH': 0.47619047619047616, 'NV': 0.8069458631256384, 'VA': 0.9274193548387097, 'IA': 0.44585987261146487, 'GA': 0.46610169491525416, 'CO': 0.9090909090909092, 'TX': 0.36402569593147754, 'ME': 0.8998988877654196, 'IN': 0.10989010989010989, 'NM': 0.9182643794147326, 'NJ': 0.9547738693467336, 'MO': 0.15317286652078776, 'OR': 0.9456740442655935, 'UT': 0.0881057268722467, 'TN': 0.0881057268722467, 'CT': 0.9547738693467336, 'AK': 0.24918743228602383, 'KY': 0.06622516556291391, 'MD': 0.963855421686747, 'MA': 0.9729187562688064, 'AR': 0.07717750826901874, 'WA': 0.9547738693467336, 'MT': 0.15317286652078776, 'SC': 0.20674646354733406, 'CA': 0.9547738693467336, 'NY': 0.9547738693467336, 'IL': 0.9547738693467336, 'WV': 0.05524861878453039, 'OK': 0.04424778761061947, 'LA': 0.099009900990099, '

## Adjust the Probabilities
Below you can make adjustments to the implied probabilities. For example, if you think Biden will never win states where he currently has less than a 40% chance, you can change `elif prob < 0.05:` to `elif prob < 0.4:`.  Alternatively, you can simply update a state's probability via: `win_prob[FL] = 0`.

In [214]:
## Optional Step ##
## If the implied probability of winning is 95% or greater, set it to 100%
## If the implied probability of winning is 5%  or less, set it to 0%
def restrict_probabilities():
    for s, prob in win_prob.items():
        if prob > 0.8:   #Tweak this
            win_prob [s] = 1.0
        elif prob < 0.5: #Tweak this
            win_prob [s] = 0
    print (win_prob)
    
## Override Democrat's chances of winning
win_prob['FL'] = 0.5
win_prob['AZ'] = 0.5
win_prob['NC'] = 0.5
win_prob['WI'] = 0.5
win_prob['MI'] = 0.5
win_prob['PA'] = 0.5
win_prob['MN'] = 0.5

restrict_probabilities()

{'WI': 0.5, 'PA': 0.5, 'FL': 0.5, 'MI': 0.5, 'AZ': 0.5, 'MN': 0.5, 'NH': 1.0, 'NC': 0.5, 'OH': 0, 'NV': 1.0, 'VA': 1.0, 'IA': 0, 'GA': 0, 'CO': 1.0, 'TX': 0, 'ME': 1.0, 'IN': 0, 'NM': 1.0, 'NJ': 1.0, 'MO': 0, 'OR': 1.0, 'UT': 0, 'TN': 0, 'CT': 1.0, 'AK': 0, 'KY': 0, 'MD': 1.0, 'MA': 1.0, 'AR': 0, 'WA': 1.0, 'MT': 0, 'SC': 0, 'CA': 1.0, 'NY': 1.0, 'IL': 1.0, 'WV': 0, 'OK': 0, 'LA': 0, 'ID': 0, 'NE': 0, 'AL': 0, 'KS': 0, 'MS': 0, 'RI': 1.0, 'HI': 1.0, 'WY': 0, 'VT': 1.0, 'DE': 1.0, 'ND': 0, 'SD': 0, 'DC': 1.0}


In [215]:
win_prob_sorted = sorted(win_prob.items())
print (win_prob_sorted)

[('AK', 0), ('AL', 0), ('AR', 0), ('AZ', 0.5), ('CA', 1.0), ('CO', 1.0), ('CT', 1.0), ('DC', 1.0), ('DE', 1.0), ('FL', 0.5), ('GA', 0), ('HI', 1.0), ('IA', 0), ('ID', 0), ('IL', 1.0), ('IN', 0), ('KS', 0), ('KY', 0), ('LA', 0), ('MA', 1.0), ('MD', 1.0), ('ME', 1.0), ('MI', 0.5), ('MN', 0.5), ('MO', 0), ('MS', 0), ('MT', 0), ('NC', 0.5), ('ND', 0), ('NE', 0), ('NH', 1.0), ('NJ', 1.0), ('NM', 1.0), ('NV', 1.0), ('NY', 1.0), ('OH', 0), ('OK', 0), ('OR', 1.0), ('PA', 0.5), ('RI', 1.0), ('SC', 0), ('SD', 0), ('TN', 0), ('TX', 0), ('UT', 0), ('VA', 1.0), ('VT', 1.0), ('WA', 1.0), ('WI', 0.5), ('WV', 0), ('WY', 0)]


In [216]:
ps_sorted = numpy.fromiter(dict(win_prob_sorted).values(), dtype=float)
print (ps_sorted)

[0.  0.  0.  0.5 1.  1.  1.  1.  1.  0.5 0.  1.  0.  0.  1.  0.  0.  0.
 0.  1.  1.  1.  0.5 0.5 0.  0.  0.  0.5 0.  0.  1.  1.  1.  1.  1.  0.
 0.  1.  0.5 1.  0.  0.  0.  0.  0.  1.  1.  1.  0.5 0.  0. ]


In [217]:
vs_sorted = numpy.fromiter ((v for _,v in sorted(electoral_votes.items())), int)    
print (vs_sorted)

[ 3  9  6 11 55  9  7  3  3 29 16  4  6  4 20 11  6  8  8 11 10  4 16 10
 10  6  3 15  3  5  4 14  5  6 29 18  7  7 20  4  9  3 11 38  6 13  3 12
 10  5  3]


In [218]:
import numpy

def simulate(n):
    rs = numpy.random.rand(n, len(win_prob_sorted))

    sims_won = 0
    for r in rs:
        wins = numpy.greater(ps_sorted , r)
        total_votes_won = numpy.sum(numpy.multiply(wins, vs_sorted))
        if total_votes_won > 269: ## Tie breaks go to republicans
            sims_won+=1

    return sims_won / n

In [219]:
simulate (100000)

0.63438

## Limitations
Nebraska and Maine uses the congressional district method to assign their votes. However, their electoral votes are assigned using the winner-take-all system in our simulations.

## Additional Resources
https://www.270towin.com/